# 歡迎光臨
與舊版本不同，此版本是為了增加彈性而生。<br>
***請注意，使用此檔案前須將所有必要的附屬檔案放到這份Code存在的資料夾中。***
1. 在Vivado中，Generate Bitstream結束後產生的檔案：<br>
    ***(兩個檔名必須改成一樣)***
    * `.hwh` 檔案，位於 `{ project_location }/{project_name}.runs/impl_1/design_1_wrapper.bit`<br>
    * `.bit` 檔案，位於 `{ project_location }/{project_name}.gen/{Design Name 1}/bd/{Design Name 2}/hw_handoff/design_1.hwh`<br><br>
2. 原先Testbench所需的測試檔 (wip)
---
## 連結
點選連結快速跳到你想看的地方
* ### [參數設定](#設定預設值) 
* ### [函數定義](#函數定義) 
* ### [主程式起點](#主程式) 
---

* ### 設定基礎路徑

In [1]:
baseDir = '/home/xilinx/jupyter_notebooks/[TOP] Flexible University Project/'

* ### 匯入所需模組

In [2]:
from pynq import Overlay
from pynq import allocate
from pynq import MMIO

import time

import ctypes
from ctypes import cdll, c_int32, c_uint, POINTER, c_char_p, c_int, c_float, c_bool, c_short

import numpy as np

import sys
np.set_printoptions(threshold=sys.maxsize)

* #### 匯入C code dll

In [3]:
# 載入 SO 動態連結檔案
ML_proccess_dll = cdll.LoadLibrary(baseDir + 'ML_proccess.so')
# 設定 SO 檔案中 sort 函數的參數資料型態
ML_proccess_dll.ML_proccess.argtypes = [POINTER(c_bool), POINTER(c_bool), POINTER(c_short), c_uint, c_uint, c_uint, c_uint, c_uint, c_uint, c_uint]
ML_proccess_dll.arr_axis_fc_modeling.argtypes = [POINTER(c_bool), c_uint, c_uint, c_uint, c_uint, POINTER(c_float), POINTER(c_float)]
ML_proccess_dll.findMAX.argtypes = [POINTER(c_float), c_int]
# 設定 SO 檔案中 sort 函數的傳回值資料型態
ML_proccess_dll.ML_proccess.restype = POINTER(c_bool)
ML_proccess_dll.arr_axis_fc_modeling.restype = POINTER(c_float)
ML_proccess_dll.findMAX.restype = c_int


* ### memory test

* ### OOP-Wire
用法：
* a = wire(name = '', width = 3, data = 1)
* 可以用slice，會回傳boolean array (ex. a[1:3])
* 直接修改Data，如a.data = 12
* 可使用cascade疊加函數組合埠(底下有範例用法)

In [4]:
import math


class Wire:
    def __init__(self, width: int = 32, data: int = 0) -> None:
        # if len(args) == 1:
        #     # Check
        #     for _ in args:
        #         if not len(_) == 2 and not type(_) == list:
        #             raise TypeError('請注意輸入格式')
        # else:
        #     raise NotImplementedError('不支援同時綁定多條線')
        if not type(width) == int and not type(data) == int:
            raise TypeError('請輸入整數')
        if width < 1 and width % 1 == 0:
            raise ValueError('寬度須為正整數')
        self._OverflowChecker(width=width, data=data)
        self._width = width
        self._data = data

    @property
    def data(self):
        return self._data

    @property
    def width(self):
        return self._width

    @data.setter
    def data(self, newVal):
        self._OverflowChecker(width=self._width, data=newVal)
        self._data = newVal

    def __getitem__(self, idx):
        # Just access like Verilog
        l = self._IntToBoolConv(data=self._data, width=self._width)
        #l_len = len(l)
        # print('int2bool', l)
        # print(self)
        if type(idx) == int:
            return Wire(data=l[idx], width=1)
        else:
            l = l[idx.stop : idx.start + 1: idx.step]
            # print(l)
            if len(l) == 0:
                raise ValueError('陣列取值Start, Stop的順序錯誤！')
            # print('bool2int', self._bool2int(l))
            return Wire(data=self._bool2int(l), width=len(l))

    # Bitwise Setter
    def __setitem__(self, idx: int, newVal: bool):
        data = self._IntToBoolConv()
        if type(idx) == int:
            data[idx] = newVal
        else: 
            w = len(data[idx.stop:idx.start + 1:idx.step])
            total_l = len(data)
            data[idx.stop:idx.start+1:idx.step] = self._IntToBoolConv(width=w, data=newVal)
            #data[idx] = self._IntToBoolConv(width=len(data[idx]), data=newVal)

        self._data = self._bool2int(data)

    def __repr__(self) -> str:
        bin_arr=self._IntToBoolConv()
        bin_arr.reverse()

        return "[{}:0] = {} > {}".format(self._width - 1, self._data, list(map(int, (bin_arr))))

    def __len__(self):
        return self._width

    def _OverflowChecker(self, width: int = None, data: int = None) -> None:
        if data is None or width is None:
            data = self._data
            width = self._width
        if math.log2(data + 1) > width:
            raise OverflowError('溢位')

    def _IntToBoolConv(self, width: int = None, data: int = None) -> list:
        if data is None or width is None:
            data = self._data
            width = self._width
        arr = ([bool(data & (1 << n)) for n in range(width)])
        #arr.reverse()
        return arr

    def _bool2int(self, bool_array: list) -> int:
        ret = 0
        for _ in range(len(bool_array)):
            ret += int(bool_array[_]) << _

        return ret

    def getIntVal(self) -> int:
        return self._data


def cascade(*wires) -> Wire:
    width = 0
    data = 0
    for wire in reversed(wires):
        if not type(wire) == Wire:
            raise TypeError()
        # print(wire.data, wire.width)
        data += wire.data << width
        width += wire.width

    return Wire(data=data, width=width)

## 函數定義
---

* ### ofmaps_padding

In [5]:
def ofmaps_padding(ifmaps_in):
    if(ifmaps_in.shape[2]%2==0):
        h_padding = np.zeros((ifmaps_in.shape[0],ifmaps_in.shape[1],1),dtype=np.bool_)
        h_padding[:,1::2].fill(1)
        ifmaps_in = np.concatenate((h_padding,ifmaps_in), axis=2) 
        h_padding = np.zeros((ifmaps_in.shape[0],ifmaps_in.shape[1],1),dtype=np.bool_)
        h_padding[:,::2].fill(1)
        ifmaps_in = np.concatenate((ifmaps_in,h_padding), axis=2) 
    else:
        h_padding = np.zeros((ifmaps_in.shape[0],ifmaps_in.shape[1],1),dtype=np.bool_)
        h_padding[:,1::2].fill(1)
        ifmaps_in = np.concatenate((h_padding,ifmaps_in,h_padding), axis=2) 

    if(ifmaps_in.shape[1]%2==0):
        v_padding = np.zeros((ifmaps_in.shape[0],1,ifmaps_in.shape[2]),dtype=np.bool_)
        v_padding[:,:,::2].fill(1)
        ifmaps_in = np.concatenate((v_padding,ifmaps_in), axis=1) 
        v_padding = np.concatenate((v_padding[:,:,1:],v_padding[:,:,:1]), axis=2)
        ifmaps_in = np.concatenate((ifmaps_in,v_padding), axis=1) 
    else:
        v_padding = np.zeros((ifmaps_in.shape[0],1,ifmaps_in.shape[2]),dtype=np.bool_)
        v_padding[:,:,::2].fill(1)
        ifmaps_in = np.concatenate((v_padding,ifmaps_in, v_padding), axis=1) 

    return ifmaps_in

* ### ifmaps modeling

In [6]:
def ifmaps_modeling(ifmaps_in,ifmaps_width,ifmaps_height,ifmaps_channel,weight_width,weight_height,stride,padding=True):
    
    if padding == True:
        ifmaps_in = ifmaps_in.reshape(ifmaps_channel,ifmaps_height-2,ifmaps_width-2)
        ifmaps_in = ofmaps_padding(ifmaps_in)
        ifmaps_in = ifmaps_in.reshape(-1)
        
    else:
        if (ifmaps_width%stride!=0):
            ifmaps_in = ifmaps_in.reshape(ifmaps_channel,ifmaps_height,ifmaps_width)
            pad_width = weight_width-(ifmaps_width%stride)
            h_padding = np.zeros((ifmaps_in.shape[0],ifmaps_in.shape[1],pad_width),dtype=np.bool_)
            ifmaps_in = np.concatenate((ifmaps_in,h_padding), axis=2) 
            v_padding = np.zeros((ifmaps_in.shape[0],pad_width,ifmaps_in.shape[2]),dtype=np.bool_)
            ifmaps_in = np.concatenate((ifmaps_in,v_padding), axis=1)  
            np.save('ifmaps5_test', ifmaps_in)
            ifmaps_in = ifmaps_in.reshape(-1)
   
    
    return ifmaps_in

## 主程式起點
---

In [7]:
####MNIST
setting_kernel_size    = [3,2,3,2,3,2]
setting_input_channel  = [1,32,32,64,64,128]
setting_ofmaps_channel = [32,32,64,64,128,128]
setting_stride         = [1,2,1,2,1,2]
setting_function       = [0,1,0,1,0,1]
setting_ifmaps_width   = [30,28,16,14,9,7]
setting_ofmaps_width   = [28,14,14,7,7,4]
setting_out_class      = 10
setting_use_fcbias     = 1  # 1: Ture  0: False  
label_name = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

setting_output_array_size = [25088,6272,12544,3136,6272,2048]


In [8]:
####MNIST
weight_npy = {}
weight_ptr = {}

weight_npy["weight0_npy"] = np.array(np.load(baseDir + 'weight0_forC.npy'))
weight_npy["weight2_npy"] = np.array(np.load(baseDir + 'weight2_forC.npy'))
weight_npy["weight4_npy"] = np.array(np.load(baseDir + 'weight4_forC.npy'))

weight_ptr["weight0_ptr"] = weight_npy["weight0_npy"].ctypes.data_as(POINTER(c_bool))
weight_ptr["weight2_ptr"] = weight_npy["weight2_npy"].ctypes.data_as(POINTER(c_bool))
weight_ptr["weight4_ptr"] = weight_npy["weight4_npy"].ctypes.data_as(POINTER(c_bool))

weight_npy["fc_weight_npy"] = np.array(np.load(baseDir + 'fc_weight.npy'))



bias_npy = {}
bias_ptr = {}

bias_npy["bias0_npy"] = np.array(np.load(baseDir + 'bias0.npy'))
bias_npy["bias2_npy"] = np.array(np.load(baseDir + 'bias2.npy'))
bias_npy["bias4_npy"] = np.array(np.load(baseDir + 'bias4.npy'))

bias_npy["bias0_npy"] = bias_npy["bias0_npy"].astype(np.int16)
bias_npy["bias2_npy"] = bias_npy["bias2_npy"].astype(np.int16)
bias_npy["bias4_npy"] = bias_npy["bias4_npy"].astype(np.int16)

bias_ptr["bias0_ptr"] = bias_npy["bias0_npy"].ctypes.data_as(POINTER(c_short))
bias_ptr["bias2_ptr"] = bias_npy["bias2_npy"].ctypes.data_as(POINTER(c_short))
bias_ptr["bias4_ptr"] = bias_npy["bias4_npy"].ctypes.data_as(POINTER(c_short))

bias_npy["fc_bias_npy"] = np.array(np.load(baseDir + 'fc_bias.npy'))


In [9]:
fc_weight_ptr = weight_npy["fc_weight_npy"].ctypes.data_as(POINTER(c_float))
fc_bias_ptr = bias_npy["fc_bias_npy"].ctypes.data_as(POINTER(c_float))

* ### Packer

In [10]:
settings = {
    'setting_kernel_size'       : setting_kernel_size,
    'setting_input_channel'     : setting_input_channel,
    'setting_ofmaps_channel'    : setting_ofmaps_channel,
    'setting_stride'            : setting_stride,
    'setting_function'          : setting_function,
    'setting_ifmaps_width'      : setting_ifmaps_width,
    'setting_ofmaps_width'      : setting_ofmaps_width,
    'setting_out_class'         : setting_out_class,
    'setting_use_fcbias'        : setting_use_fcbias,
    'label_name'                : label_name,
    'setting_output_array_size' : setting_output_array_size
}


In [11]:
def main_code(array_from_front_end, settings=settings, weight_npy=weight_npy, bias_npy=bias_npy, fc_weight_ptr=fc_weight_ptr, fc_bias_ptr=fc_bias_ptr):
    
    conv_times = 0 
    pool_times = 0 
    fc_times   = 0
    data_prep_times = 0
    total_start_time = time.time()
    
    # Workround - Data Unpack
    setting_kernel_size = settings['setting_kernel_size']
    setting_input_channel = settings['setting_input_channel']
    setting_ofmaps_channel = settings['setting_ofmaps_channel']
    setting_stride = settings['setting_stride']
    setting_function = settings['setting_function']
    setting_ifmaps_width = settings['setting_ifmaps_width']
    setting_ofmaps_width = settings['setting_ofmaps_width']
    setting_out_class = settings['setting_out_class']
    setting_use_fcbias = settings['setting_use_fcbias']
    label_name = settings['label_name']
    setting_output_array_size = settings['setting_output_array_size']
        
    ifmaps_arr = np.array(array_from_front_end)
    
    for now_layer in range(6):


        if(now_layer != 0):
            data_prep_start_time = time.time()
            if(setting_function[now_layer] == 0):
                padding_en = True
            else:
                padding_en = False
            
            ifmaps_arr = ifmaps_modeling(C_PL_ret_arr,setting_ifmaps_width[now_layer],
                                         setting_ifmaps_width[now_layer],setting_input_channel[now_layer],
                                         setting_kernel_size[now_layer],setting_kernel_size[now_layer],
                                         setting_stride[now_layer],padding_en)
            data_prep_end_time = time.time()
            data_prep_times += data_prep_end_time - data_prep_start_time
            
        C_PL_ifmaps_in_ptr = ifmaps_arr.ctypes.data_as(POINTER(c_bool))
        
        if(setting_function[now_layer] == 0):
            conv_start_time = time.time()
            
            C_PL_ret_ptr = ML_proccess_dll.ML_proccess(C_PL_ifmaps_in_ptr,weight_ptr["weight{}_ptr".format(now_layer)],
                                                     bias_ptr["bias{}_ptr".format(now_layer)], setting_kernel_size[now_layer],
                                                     setting_input_channel[now_layer],setting_ofmaps_channel[now_layer],
                                                     setting_stride[now_layer],setting_function[now_layer],
                                                     setting_ifmaps_width[now_layer],setting_ofmaps_width[now_layer])
            
            # 將回傳指標指定大小及型態  [0]<-不懂用意
            C_PL_ret_ptr_cast = ctypes.cast(C_PL_ret_ptr, ctypes.POINTER(ctypes.c_bool*setting_output_array_size[now_layer]))[0]

            # 將buffer轉換成numpy array
            C_PL_ret_arr = np.frombuffer(C_PL_ret_ptr_cast, np.bool_)
            
            conv_end_time = time.time()
            conv_times += conv_end_time - conv_start_time
        else:
            pool_start_time = time.time()
            
            if(setting_ifmaps_width[now_layer]%setting_stride[now_layer]!=0):
                ifmaps_width_pool = setting_ifmaps_width[now_layer] + setting_kernel_size[now_layer]-(setting_ifmaps_width[now_layer]%setting_stride[now_layer])


            else:
                ifmaps_width_pool = setting_ifmaps_width[now_layer]
            
            C_PL_ret_ptr = ML_proccess_dll.ML_proccess(C_PL_ifmaps_in_ptr,None,
                                                     None, setting_kernel_size[now_layer],
                                                     setting_input_channel[now_layer],setting_ofmaps_channel[now_layer],
                                                     setting_stride[now_layer],setting_function[now_layer],
                                                     ifmaps_width_pool,setting_ofmaps_width[now_layer])
            
            # 將回傳指標指定大小及型態  [0]<-不懂用意
            C_PL_ret_ptr_cast = ctypes.cast(C_PL_ret_ptr, ctypes.POINTER(ctypes.c_bool*setting_output_array_size[now_layer]))[0]

            # 將buffer轉換成numpy array
            C_PL_ret_arr = np.frombuffer(C_PL_ret_ptr_cast, np.bool_)
            
            pool_end_time = time.time()
            pool_times += pool_end_time - pool_start_time
            
    
    fc_start_time = time.time()
    
    # 呼叫 SO 檔案中的 FC 函數
    C_PL_data_ptr = C_PL_ret_arr.ctypes.data_as(POINTER(c_bool))  # TODO: 這個拖慢了時間 占了 0.001多秒
#     print(C_PL_ret_arr)
    # output after solfmax
    answer_arr_ptr = ML_proccess_dll.arr_axis_fc_modeling(C_PL_data_ptr,
                                                setting_ofmaps_channel[5]*setting_ofmaps_width[5]*setting_ofmaps_width[5],
                                                setting_out_class, setting_ofmaps_width[5],
                                                setting_use_fcbias,
                                                fc_weight_ptr,fc_bias_ptr)
    
    answer = ML_proccess_dll.findMAX(answer_arr_ptr, setting_out_class)
    
    answer_arr_ptr_cast = ctypes.cast(answer_arr_ptr, ctypes.POINTER(c_float*setting_out_class))[0] # TODO: 這個拖慢了時間 占了 0.005多秒

    # 將buffer轉換成numpy array
    answer_arr = np.frombuffer(answer_arr_ptr_cast, np.float32) # TODO: 這個拖慢了時間 占了 0.001多秒
    
    total_end_time = time.time()
    fc_times = total_end_time - fc_start_time
    total_times = (total_end_time-total_start_time)
    return {'result': label_name[answer] ,'probability_arr': answer_arr.tolist() ,'total_time': total_times, 
            'data_prep_time': data_prep_times ,'conv_time':conv_times ,'pool_time':pool_times ,'fc_time':fc_times}

if __name__ == "__main__":
    print(main_code(np.loadtxt(baseDir + 'ifmaps0.txt', dtype=np.bool_)))  # 正常一般就會執行一次Main Code，不用在外面呼叫

{'result': '7', 'probability_arr': [1.2477482380646165e-09, 3.8720452266716165e-07, 2.6563263872958487e-06, 1.2272469973595435e-07, 1.5914446294118534e-07, 4.4833846857272874e-08, 2.8254939620636632e-12, 0.9999939203262329, 1.603923660553619e-09, 2.769385446299566e-06], 'total_time': 0.1527082920074463, 'data_prep_time': 0.004103183746337891, 'conv_time': 0.14418816566467285, 'pool_time': 0.0023789405822753906, 'fc_time': 0.0010459423065185547}


In [17]:
total_times = 0
for x in range(50):
    ret_dic = main_code( np.loadtxt(baseDir + 'ifmaps0.txt', dtype=np.bool_))
    total_time = ret_dic['total_time']
    total_times += total_time
    print(total_time)
avg_time = total_times/50
print('avg time = ', avg_time)
print('fps = ', 1/avg_time)

0.14084482192993164
0.13959693908691406
0.14013361930847168
0.13973760604858398
0.13984251022338867
0.14005279541015625
0.14002752304077148
0.14026737213134766
0.1400916576385498
0.1393904685974121
0.14021682739257812
0.13919734954833984
0.13971614837646484
0.13953423500061035
0.14025259017944336
0.14005446434020996
0.139906644821167
0.13961052894592285
0.13999199867248535
0.13965582847595215
0.1397244930267334
0.13953709602355957
0.14050602912902832
0.1400146484375
0.14018034934997559
0.13997507095336914
0.14047908782958984
0.13906407356262207
0.14014053344726562
0.13913178443908691
0.14070367813110352
0.13985991477966309
0.1405479907989502
0.14042139053344727
0.1409604549407959
0.13880228996276855
0.139817476272583
0.13967394828796387
0.1398005485534668
0.13922595977783203
0.13938450813293457
0.13912582397460938
0.14041686058044434
0.13919997215270996
0.13998794555664062
0.13985371589660645
0.14035272598266602
0.1397101879119873
0.1400439739227295
0.13951826095581055
avg time =  0.13

In [12]:
# def main_code(array_from_front_end, AXI_wires=AXI_wires, settings=settings, weight_npy=weight_npy, bias_npy=bias_npy, output_data=output_data, fc_weight_ptr=fc_weight_ptr, fc_bias_ptr=fc_bias_ptr):
    
#     conv_times = 0 
#     pool_times = 0 
#     fc_times   = 0
#     data_prep_times = 0
#     total_start_time = time.time()
    
#     # Workround - Data Unpack
#     setting_kernel_size = settings['setting_kernel_size']
#     setting_input_channel = settings['setting_input_channel']
#     setting_ofmaps_channel = settings['setting_ofmaps_channel']
#     setting_stride = settings['setting_stride']
#     setting_function = settings['setting_function']
#     setting_ifmaps_width = settings['setting_ifmaps_width']
#     setting_ofmaps_width = settings['setting_ofmaps_width']
#     setting_out_class = settings['setting_out_class']
#     setting_use_fcbias = settings['setting_use_fcbias']
#     label_name = settings['label_name']
#     setting_output_array_size = settings['setting_output_array_size']
        
#     array_from_front_end = np.array(array_from_front_end)
    
#     for now_layer in range(6):


#         if(now_layer != 0):

#             if(setting_function[now_layer] == 0):
#                 padding_en = True
#             else:
#                 padding_en = False

#             ifmaps_arr = ifmaps_modeling(output_data[:setting_output_array_size[now_layer-1]],setting_ifmaps_width[now_layer],
#                                          setting_ifmaps_width[now_layer],setting_input_channel[now_layer],
#                                          setting_kernel_size[now_layer],setting_kernel_size[now_layer],
#                                          setting_stride[now_layer],padding_en)

#     # 呼叫 SO 檔案中的 FC 函數
#     output_data_ptr = output_data.ctypes.data_as(POINTER(c_uint))  # TODO: 這個拖慢了時間 占了 0.001多秒
    
#     # output after solfmax
#     answer_arr_ptr = arr_axis_modeling_dll.arr_axis_fc_modeling(output_data_ptr,
#                                                 setting_ofmaps_channel[5]*setting_ofmaps_width[5]*setting_ofmaps_width[5],
#                                                 setting_out_class, setting_ofmaps_width[5],
#                                                 setting_use_fcbias,
#                                                 fc_weight_ptr,fc_bias_ptr)
#     answer = arr_axis_modeling_dll.findMAX(answer_arr_ptr, setting_out_class)
    
#     answer_arr_ptr_cast = ctypes.cast(answer_arr_ptr, ctypes.POINTER(c_float*setting_out_class))[0] # TODO: 這個拖慢了時間 占了 0.005多秒

#     # 將buffer轉換成numpy array
#     answer_arr = np.frombuffer(answer_arr_ptr_cast, np.float32) # TODO: 這個拖慢了時間 占了 0.001多秒

#     return {'result': label_name[answer] ,'probability_arr': answer_arr.tolist() ,'total_time': total_times, 
#             'data_prep_time': data_prep_times ,'conv_time':conv_times ,'pool_time':pool_times ,'fc_time':fc_times}


# if __name__ == "__main__":
#     print(main_code(np.loadtxt(baseDir + 'ifmaps0.txt', dtype=np.uint32)))  # 正常一般就會執行一次Main Code，不用在外面呼叫